In [27]:
import pymupdf
import os
import pandas as pd
import re
import time

folder_path = 'data/'

# Getting the list of all pdfs in the folder
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Table

In [28]:
def split_text_on_alphabetic_start(text):
    lines = text.splitlines()

    blocks = []
    current_block = []

    for line in lines:
        # Checking if the first word of the line is alphabetic
        first_word = line.strip()[0]
        if first_word.isalpha():
            # If current_block is not empty, adding it to the list of blocks
            if current_block:
                blocks.append("\n".join(current_block))
                current_block = []  # Resetting the current block
        # Adding the current line to the ongoing block
        current_block.append(line)

    # Adding the last block to the list if not empty
    if current_block:
        blocks.append("\n".join(current_block))

    return blocks
        
def join_consecutive_alphabetic_lines(text):
    lines = text.splitlines()

    i = 0
    # Looping through the lines, checing consecutive pairs
    while i < len(lines) - 1:
        # Checking if both current and next line contain alphabets
        if any(char.isalpha() for char in lines[i]) and any(char.isalpha() for char in lines[i + 1]):
            # Joining the next line to the current line
            lines[i] = lines[i] + "" + lines[i + 1]
            # Remove the next line
            del lines[i + 1]
        else:
            # Moving to the next line
            i += 1

    # Joining the modified list back into a single string
    return "\n".join(lines)

In [29]:
def Table_df_extractor(text):
    table_part = text.split('#\n')[1].split("1\n", maxsplit=1)[1].split('Taxable Amount')[0]

    Item_list = []
    Rate_Item_list = []
    Qty_list = []
    Taxable_Value_list = [] = []
    Tax_Amount_list = []
    Amount_list = []

    input_text = join_consecutive_alphabetic_lines(table_part)

    # Splitting the text based on the alphabetic start condition
    blocks = split_text_on_alphabetic_start(input_text)

    for i in range(len(blocks)):
        if i!=len(blocks)-1:
            lines = blocks[i].splitlines()

            # Removing the last line if it exists
            if lines:
                lines = lines[:-1]

            # Joining the remaining lines back into a single string
            blocks[i] = "\n".join(lines)

            lines = blocks[i].splitlines()

            # Repeating the process until the number of lines is 6
            while len(lines) > 6:
                # Concatenating the 2nd and 3rd lines
                lines[1] = lines[1] + " = " + lines[2]
                # Remove the 3rd line
                del lines[2]

            # Joining the remaining lines back into a single string
            blocks[i] = "\n".join(lines)

        else:
            lines = blocks[i].splitlines()

            # Repeating the process until the number of lines is 6
            while len(lines) > 6:
                # Concatenating the 2nd and 3rd lines
                lines[1] = lines[1] + " = " + lines[2]
                # Removeing the 3rd line
                del lines[2]

            # Joining the remaining lines back into a single string
            blocks[i] = "\n".join(lines)


        Item_list.append(lines[0])
        Rate_Item_list.append(lines[1])
        Qty_list.append(lines[2])
        Taxable_Value_list.append(lines[3])
        Tax_Amount_list.append(lines[4])
        Amount_list.append(float(lines[5].replace(",", "")))

    Table_df = pd.DataFrame({'Item_list':Item_list,
                             'Rate_Item_list':Rate_Item_list,
                             'Qty_list':Qty_list,
                             'Taxable_Value_list':Taxable_Value_list,
                             'Tax_Amount_list':Tax_Amount_list,
                             'Amount_list':Amount_list})
    return Table_df

# Tax

In [30]:
def extract_tax_text(text):
    lines = text.splitlines()
    
    # Initializing a list to hold the tax details
    tax_details = []
    
    in_tax_details = False
    
    for line in lines:
        # Checking if we reached the start of the tax details section
        if "Taxable Amount" in line:
            in_tax_details = True
            
        # Checking if we reached the end of the tax details section
        if in_tax_details:
            if line.strip().startswith("Total Items"):
                break
            tax_details.append(line.strip())
    
    return "\n".join(tax_details)

def parse_text_to_dataframe(text):
    input_text = extract_tax_text(text)
    lines = input_text.splitlines()

    data = []

    for i in range(0, len(lines) - 1, 2):
        entity = lines[i].strip()
        value_str = lines[i + 1].strip()

        # Removing the currency symbol and convert to float
        value_float = float(''.join(char for char in value_str if (char.isdigit() or char=='.')))

        # Appending the tuple to the data list
        data.append((entity, value_float))

    # Creating a DataFrame from the data list
    df = pd.DataFrame(data, columns=["Entity", "Value"])
    return df

# Misc Data

In [31]:
def get_misc_data(text):
    lines = text.splitlines()

    info = {}

    # Extracting relevant fields using loops and conditions of having ":"
    for i, line in enumerate(lines):

        if (":" in line) and (line.strip()[-1] == ":") and (lines[i + 1].strip()[-1] != ":"):
            info[line.strip()[:-1]] = lines[i + 1].strip()
        
        if (":" in line) and (line.strip()[-1] != ":") and (len(line.strip().split('/'))>=3):
            info[line.strip().split(':')[0].split('/')[0].strip()] = line.split(":")[-1].split('/')[0].strip()
            info[line.strip().split(':')[0].split('/')[1].strip()] = line.split(":")[-1].split('/')[1].strip()
        
        if (":" in line) and (line.strip()[-1] != ":") and (len(line.strip().split('/'))<3):
            info[line.strip().split(':')[0]] = line.split(":")[-1].strip()
        
        if "GSTIN" in line:
            gstin_match = re.search(r"GSTIN\s+(\S+)", line)
            if gstin_match:
                info["GSTIN"] = gstin_match.group(1)

        if "GSTIN" in line:
            address_lines = []
            j = i + 1
            while "Mobile" not in lines[j]:
                address_lines.append(lines[j])
                j += 1
            info["Address"] = ", ".join(address_lines)

        if "Mobile" in line:
            mobile_match = re.search(r"Mobile\s+(\+\d+\s\d+)", line)
            if mobile_match:
                info["Mobile"] = mobile_match.group(1)

        email_match = re.search(r"Email\s+(\S+@\S+)", line)
        if email_match:
            info["Email"] = email_match.group(1)

    df = pd.DataFrame(list(info.items()), columns=["Entity", "Data"])

    return df

# Validity Check

In [32]:
# Validation functions
def is_sum_amount_table_round_off(Table_df, Tax_df):
    try:
        if any(Tax_df['Entity'].str.lower() == 'discount'):
            result = round(sum(Table_df['Amount_list']) - Tax_df.loc[Tax_df['Entity'].str.lower() == 'discount', 'Value'].values[0], 0) == round(Tax_df.loc[Tax_df['Entity'].str.lower() == 'total', 'Value'].values[0],0)
        else:
            result = round(sum(Table_df['Amount_list']), 0) == round(Tax_df.loc[Tax_df['Entity'].str.lower() == 'total', 'Value'].values[0],0)
        return result
    except:
        return False
    
def is_sum_amount_tax(df):
    try:
        taxable_amount = df.loc[df['Entity'].str.lower() == "taxable amount", "Value"].values[0]
        total = df.loc[df['Entity'].str.lower() == "total", "Value"].values[0]

        # Calculating the sum by adding applicable values and subtracting "Round Off" (if available)
        additional_sum = df[(~df['Entity'].str.lower().isin(["taxable amount", "round Off", "total", "total discount", "discount"]))]['Value'].sum()

        # Subtracting the "Round Off" value if present
        round_off_value = df.loc[df['Entity'].str.lower() == "round off", "Value"].sum()
        discount_value = df.loc[df['Entity'].str.lower() == "discount", "Value"].sum()

        calculated_total = taxable_amount + additional_sum - round_off_value - discount_value

        # Checking if the calculated total matches the given "Total"
        is_valid = round(calculated_total,0) == round(total,0)
        return is_valid
    except:
        return False
    
# Checking misc data
def is_valid_total_unique_qty(Table_df, misc_data_df):
    try:
        uniq = len(Table_df)
        temp = float(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'total items', 'Data'].values[0])
        return uniq == temp
    except:
        return False
    
def is_valid_total_qty(Table_df, misc_data_df):
    try:
        qty_sum = Table_df['Qty_list'].str.extract(r'(\d+)').astype(float).sum()[0]
        temp = float(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'qty', 'Data'].values[0])
        return qty_sum == temp
    except:
        return False


def is_valid_mobile(mobile):
    try:
        return bool(re.match(r'^\+91\s[0-9]{10}$', mobile))
    except:
        return False

def is_valid_email(email):
    try:
        return "@" in email and "." in email
    except:
        return False

def is_valid_invoice_number(invoice_number):
    try:
        return invoice_number.startswith("INV-")
    except:
        return False

def is_valid_date(date):
    try:
        pd.to_datetime(date, format="%d %b %Y")
        return True
    except ValueError:
        return False

def is_valid_quantity(qty):
    try:
        float_qty = float(qty)
        return float_qty > 0
    except ValueError:
        return False

def is_valid_account_number(account_number):
    try:
        return account_number.isdigit()
    except:
        return False

In [33]:
def validity_checker_df(Table_df, Tax_df, misc_data_df):
    Validity_df = pd.DataFrame({'mobile':[is_valid_mobile(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'mobile', 'Data'].values[0])],
                            'email':[is_valid_email(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'email', 'Data'].values[0])],
                            'invoice_number':[is_valid_invoice_number(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'invoice #', 'Data'].values[0])],
                            'invoice_date':[is_valid_date(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'invoice date', 'Data'].values[0])],
                            'due_date':[is_valid_date(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'due date', 'Data'].values[0])],
                            'total_items':[is_valid_quantity(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'total items', 'Data'].values[0])],
                            'qty':[is_valid_quantity(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'qty', 'Data'].values[0])],
                            'account_number':[is_valid_account_number(misc_data_df.loc[misc_data_df['Entity'].str.lower() == 'account #', 'Data'].values[0])],
                            'sum_amount_table_round_off':[is_sum_amount_table_round_off(Table_df, Tax_df)],
                            'sum_amount_tax':[is_sum_amount_tax(Tax_df)],
                            'total_unique_quantity':[is_valid_total_unique_qty(Table_df, misc_data_df)],
                            'total_quantity':[is_valid_total_qty(Table_df, misc_data_df)]})
    return Validity_df

# Full Run

In [34]:
Overall_validity_df_list = []

output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

    
# Starting time
start_time = time.time()

for pdf in file_names:
    try:
        doc = pymupdf.open(f"data/{pdf}") 
        for page in doc:
            text = page.get_text()
            
        Table_df = Table_df_extractor(text)
        Tax_df = parse_text_to_dataframe(text)
        Misc_df = get_misc_data(text)
        
        Validity_df = validity_checker_df(Table_df, Tax_df, Misc_df)
        Overall_validity_df_list.append(Validity_df)
        
        file_name = f'{output_dir}/{pdf.split(".")[0]}.xlsx'

        # Using ExcelWriter to save all DataFrames to different sheets in same file
        with pd.ExcelWriter(file_name) as writer:
            Table_df.to_excel(writer, sheet_name='Medicine Bill', index=False)
            Misc_df.to_excel(writer, sheet_name='Personal Information', index=False)
            Tax_df.to_excel(writer, sheet_name='Tax and Total Amount', index=False)
            Validity_df.to_excel(writer, sheet_name='Data Validity', index=False)

        print(f'Successfully Extracted Data From {pdf}')
        
    except:
        print(f"\n\n\nERROR ==================== {pdf}\n\n\n")
        
Overall_validity_df = pd.concat(Overall_validity_df_list, axis=0, ignore_index=True)     
        
# Ending time
end_time = time.time()

Successfully Extracted Data From INV-117_Naman.pdf
Successfully Extracted Data From INV-118_Rashu.pdf
Successfully Extracted Data From INV-121_Jitesh Soni.pdf
Successfully Extracted Data From INV-123_Asit.pdf
Successfully Extracted Data From INV-124_Ankita Sattva.pdf
Successfully Extracted Data From INV-127_Avik Mallick.pdf
Successfully Extracted Data From INV-128_Atia Latif.pdf
Successfully Extracted Data From INV-129_Divya Suhane.pdf
Successfully Extracted Data From INV-133_Sheetal Kapur.pdf
Successfully Extracted Data From INV-134_Sheetal Kapur.pdf
Successfully Extracted Data From INV-135_Mohith Saragur.pdf
Successfully Extracted Data From INV-136_Rishabh Ramola.pdf
Successfully Extracted Data From INV-138_Agrani Kandele.pdf
Successfully Extracted Data From INV-140_Ankit.pdf
Successfully Extracted Data From INV-141_Kasturi Kalwar.pdf
Successfully Extracted Data From INV-142_Urmila Jangam.pdf
Successfully Extracted Data From INV-143_Prashant.pdf
Successfully Extracted Data From INV-1

In [35]:
# Calculating the time taken
time_taken = end_time - start_time
print(f"Time taken: {time_taken:.6f} seconds")

Time taken: 1.419003 seconds


# Accuracy Analysis

In [36]:
Overall_validity_df

,mobile,email,invoice_number,invoice_date,due_date,total_items,qty,account_number,sum_amount_table_round_off,sum_amount_tax,total_unique_quantity,total_quantity
0,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True


In [37]:
# Calculating accuracy percentages of each entity
accuracy_percentages = {}

for column in Overall_validity_df.columns:
    true_count = Overall_validity_df[column].sum()
    total_count = len(Overall_validity_df)
    accuracy = (true_count / total_count) * 100
    accuracy_percentages[column] = round(accuracy, 2)

print("Accuracy Percentages:")
for column, accuracy in accuracy_percentages.items():
    print(f"{column}: {accuracy}%")

Accuracy Percentages:
mobile: 100.0%
email: 100.0%
invoice_number: 100.0%
invoice_date: 100.0%
due_date: 100.0%
total_items: 100.0%
qty: 100.0%
account_number: 100.0%
sum_amount_table_round_off: 100.0%
sum_amount_tax: 91.67%
total_unique_quantity: 100.0%
total_quantity: 100.0%


In [38]:
Overall_accuracy_score = ((Overall_validity_df.sum().sum())*100)/(len(Overall_validity_df)*len(list(Overall_validity_df)))

print('Overall_accuracy_score: ', Overall_accuracy_score)

Overall_accuracy_score:  99.30555555555556


In [39]:
# End of file